In [9]:
import pandas as pd
import numpy as np
import random 
import re 
import os 
import warnings
warnings.filterwarnings("ignore")
import category_encoders as ce 

from sklearn.metrics import (    accuracy_score,    confusion_matrix,f1_score,precision_score, recall_score,roc_auc_score)
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold,cross_val_score, StratifiedShuffleSplit

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE , ADASYN

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,ExtraTreesClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier,AdaBoostClassifier
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import optuna
from optuna.samplers import TPESampler

from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder

from sklearn.feature_selection import RFE, RFECV
import matplotlib.pyplot as plt


In [10]:
pd.set_option('display.max_rows', 500)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

nation_corp = {
    'Austria': ['LGEAG'],    'Czech Republic': ['LGECZ'],    'France': ['LGEFS'],    'Germany': ['LGEDG'],    'Greece': ['LGEHS'],    'Hungary': ['LGEMK'],    'Italy': ['LGEIS'],    'Netherlands': ['LGESC', 'LGEEH', 'LGEBN'],    'Poland': ['LGEWR', 'LGEPL', 'LGEMA'],    'Portugal': ['LGEPT','LGEBT'],
    'EUs': ['LGEEB'],    'Romania': ['LGERO'],    'Spain': ['LGEES'],    'Sweden': ['LGENO', 'LGESW'],    'United Kingdom': ['LGEUK'],      'Kazakhstan': ['LGEAK'],    'Russia': ['LGERM', 'LGERI', 'LGERA'],
    'Ukraine': ['LGEUR'],    'Latvia': ['LGELV','LGELA'],    'Algeria': ['LGEAS'],
    'Egypt': ['LGEEG'],    'Jordan': ['LGELF'],    'Kenya': ['LGESK','LGEEF'],    'Morocco': ['LGEMC'],
    'Saudi Arabia': ['LGESJ'],    'Iran':['LGEIR'],     'Israel':['LGEYK'],     'The Republic of South Africa': ['LGESA'],
    'Tunisia': ['LGETU'],    'U.A.E': ['LGEOT', 'LGEDF', 'LGEGF', 'LGEME', 'LGEAF'],    'Nigeria': ['LGEAO', 'LGENI'],
    'Turkey': ['LGETK', 'LGEAT'],    'Australia': ['LGEAP'],
    'China': ['LGEQA', 'LGETL', 'LGECH', 'LGEYT', 'LGETR', 'LGETA', 'LGESY', 'LGESH', 'LGEQH', 'LGEQD', 'LGEPN', 'LGEND', 'LGEKS', 'LGEHZ', 'LGEHN', 'LGEHK'],
    'India': ['LGEIL'],    'Indonesia': ['LGEIN'],    'Japan': ['LGEJP'],    'Malaysia': ['LGEML'],    'Philippines': ['LGEPH'],
    'Singapore': ['LGESL'],    'Taiwan': ['LGETT'],    'Korea' :['LGEKR'],    'Thailand': ['LGETH'],    'Vietnam': ['LGEVN','LGEVH'],
     'Canada': ['LGECI'],    'Mexico': ['LGERS', 'LGEMX', 'LGEMS', 'LGEMM'],    'United States': ['LGEMR', 'LGEUS', 'LGEMU', 'LGEAI'],
    'Argentina': ['LGEAG','LGEAR'],    'Brazil': ['LGEBR','LGESP'],    'Chile': ['LGECL'],    'Colombia': ['LGEVZ', 'LGECB'],
    'Panama': ['Guatemala', 'LGEPS'],    'Peru': ['LGEPR']}
continent_nation={
    'Europe':['EUs','Austria', 'Czech Republic' ,'France' ,'Germany', 'Greece' ,'Hungary', 'Italy', 'Netherlands' ,'Poland' ,'Portugal' ,'Romania', 'Spain' ,'Sweden','United Kingdom'], 
    'Russia and CIS':['Kazakhstan','Russia', 'Ukraine', 'Latvia'],     'Africa and MiddleEast': ['Israel','Iran','Algeria', 'Egypt', 'Jordan', 'Kenya', 'Morocco','Saudi Arabia','The Republic of South Africa','Tunisia', 'U.A.E', 'Nigeria', 'Turkey'], 
    'Asia':['Korea','Australia','China','India','Indonesia','Japan','Malaysia','Philippines','Singapore','Taiwan','Thailand','Vietnam'], 
    'NorthAmerica' : ['Canada','Mexico','United States'],    'SouthAmerica' :['Argentina','Brazil','Chile','Colombia','Panama','Peru']
    
}
hemisphere = {
    'Northern': ['EUs', 'Austria', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Spain', 'Sweden', 'United Kingdom', 'Kazakhstan', 'Russia', 'Ukraine', 'Latvia', 'Israel', 'Iran', 'Jordan', 'Morocco', 'Saudi Arabia', 'Tunisia', 'Turkey', 'Korea', 'China', 'Japan', 'Taiwan', 'Canada', 'United States', 'Mexico', 'Panama'],
    'Southern': ['Algeria', 'Egypt', 'Kenya', 'The Republic of South Africa', 'U.A.E', 'Nigeria', 'Australia', 'India', 'Indonesia', 'Malaysia', 'Philippines', 'Singapore', 'Thailand', 'Vietnam', 'Argentina', 'Brazil', 'Chile', 'Colombia', 'Peru']
}
mapping_dict = {
    "others": "Other",
    "Others": "Other",
    "other_": "Other",
    "other": "Other",
    "Etc.": "ETC.",
}


In [11]:
# 데이터 생성 및 전처리 함수 
def get_datas():
    train = pd.read_csv("train.csv")
    test = pd.read_csv("submission.csv").drop(['id','is_converted'], axis =1) # 테스트 데이터(제출파일의 데이터)
    train['is_converted']=np.where(train['is_converted']==True,1,0)
    return train, test 


def delete_cols(data, cols):
    data = data.drop(columns=cols)
    return data

def log_transform(data,cols):
    for col in cols :
        data[col+'log']=np.log1p(data[col]) 
    return data 


def eda_expected_timeline(df):
    
    def timeline_label(time):
    
        time = str(time).lower().replace(' ','').replace('_','').replace('/','').replace(',','').replace('~','').replace('&','').replace('-','').replace('.','')
        
        if time == 'lessthan3months':
            result = 'less than 3 months'
        elif time == '3months6months':
            result = '3 months ~ 6 months'
        elif time == '6months9months':
            result = '6 months ~ 9 months'
        elif time == '9months1year':
            result = '9 months ~ 1 year'
        elif time == 'morethanayear':
            result = 'more than a year'
        else:
            result = 'aimers_0203'
            
        return result
    
    df['expected_timeline'] = df['expected_timeline'].apply(timeline_label)
    
    return df


# inquiry type 전처리하기 
def eda_inquiry_type(df):
    df['inquiry_type']= df['inquiry_type'].map(mapping_dict).fillna(train['inquiry_type'])
    df.loc[df['inquiry_type'].str.contains('Solicito apoyo para realizar', na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    replacement = {'/': ' ', '-':' ', '_':' '}
    df['inquiry_type'].replace(replacement, regex=True, inplace=True)
    return df


#customer type 처리 
def customer_type(data):
    data['customer_type']=data['customer_type'].fillna('none') 
    return data

# total_area 변수로 통일
def eda_business_area(df):
    for col in ['business_area','business_subarea']:
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace(" ", "") 
        df[col] = df[col].str.replace(r'[^\w\s]', "") 
        df[col] = df[col].fillna('nan') 
    df['total_area'] = df['business_area'].astype(str) + df['business_subarea'].astype(str)
    return df 

# 새로운 국가명, 대륙 열을 만들기 
def get_nation_continent(df):
    nation_corp_reverse ={v:k for k , values in nation_corp.items() for v in values }
    df['nation']=df['response_corporate'].map(nation_corp_reverse)
    continent_nation_reverse ={v:k for k , values in continent_nation.items() for v in values }
    df['continent']=df['nation'].map(continent_nation_reverse)
#     df = df.drop('customer_country',axis=1) 
    return df 

#라벨 인코딩 
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}
    series = series.astype(str)
    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)
    return series

# com_reg_ver_win_rate 최빈값으로 채우기 
def com_reg_fill(train,test):
    train['com_reg_ver_win_rate'] = train['com_reg_ver_win_rate'].fillna(train['com_reg_ver_win_rate'].mode()[0])
    test['com_reg_ver_win_rate'] = test['com_reg_ver_win_rate'].fillna(train['com_reg_ver_win_rate'].mode()[0])
    return train,test

def timeline_tonumber(row):
    if row['expected_timeline'] == 'less than 3 months':
        return int(1)
    elif row['expected_timeline'] == '3 months ~ 6 months':
        return int(3)
    elif row['expected_timeline'] == '6 months ~ 9 months':
        return int(6)
    elif row['expected_timeline'] == '9 months ~ 1 year':
        return int(9)    
    elif row['expected_timeline'] =='aimer_0203':
        return np.nan
    else : 
        return int(12)
    
def create_grouped_features(train, test, group, numeric_var):
    # 범주형 특성들에 대해서 다른 수치형 데이터의 중앙값, 최대, 합을 새로운 열로 추가하기 
    train = train.copy()
    test = test.copy()
    aggs = ['median', 'max','sum']
    for agg in aggs:
        # groupby 후 aggregation
        a1 = train.groupby([group])[numeric_var].agg(agg).to_dict()
        # 새로운 feature 생성
        train[numeric_var+'_'+group+'_'+agg] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+agg] = test[group].map(a1)
    return train, test

def do_scale(train,test, scale_cols) :
    for c in scale_cols:
        min_value = train[c].min()
        max_value = train[c].max()
        train[c] = (train[c] - min_value) / (max_value - min_value)
        test[c] = (test[c] - min_value) / (max_value - min_value)
    return train,test

In [12]:
groups = ['business_unit','customer_idx']
numeric_vars = ['historical_existing_cnt', 'lead_desc_length']
scale_cols = ['com_reg_ver_win_rate','historical_existing_cnt', 'lead_desc_length','ver_win_rate_x'] 

In [25]:
# data 갖고오기 
train,test= get_datas() 

# 스케일링 하기 
train,test =do_scale(train,test,scale_cols)
# 범주형 데이터에 대해 수치형 데이터 통계값 추가
for group in groups:
    for numeric_var in numeric_vars:
        train, test = create_grouped_features(train, test, group, numeric_var)
        
# 전처리, 로그변환 수행하기 
columns_to_log=['com_reg_ver_win_rate','lead_desc_length']
train,test= log_transform(train,columns_to_log ),log_transform(test,columns_to_log)
train,test =eda_business_area(train),eda_business_area(test)
train,test= get_nation_continent(train),get_nation_continent(test)
train,test=eda_expected_timeline(train) ,eda_expected_timeline(test)
train,test=customer_type(train), customer_type(test)
train,test=eda_inquiry_type(train), eda_inquiry_type(test) 

for col in ['customer_idx','customer_type',]:
    train[col+'count'] =train[col].map(train[col].value_counts())
    test[col+'count'] =test[col].map(train[col].value_counts())
    

train['idx_unit'] = train['customer_idx'].astype(str)+train['business_unit'].astype(str)
test['idx_unit'] = test['customer_idx'].astype(str)+test['business_unit'].astype(str)
train['idx_posi'] = train['customer_idx'].astype(str)+train['customer_position'].astype(str)
test['idx_posi'] = test['customer_idx'].astype(str)+test['customer_position'].astype(str)
train['conti_inquiry'] = train['continent'].astype(str)+train['inquiry_type'].astype(str)
test['conti_inquiry'] = test['continent'].astype(str)+test['inquiry_type'].astype(str)
# train['job_unit'] = train['business_unit'].astype(str)+train['customer_job'].astype(str)
# test['job_unit'] = test['business_unit'].astype(str)+test['customer_job'].astype(str)
#0.717 나옴 4개 변수 

In [26]:
# country.1지우지말기 
columns_to_delete=['customer_country']
# columns_to_delete=[]
train,test =delete_cols(train, columns_to_delete), delete_cols(test,columns_to_delete)

cols = [     'customer_country',    "business_subarea",    "business_area",    "business_unit",    "customer_type",    "enterprise",    "customer_job",    "inquiry_type",    "product_category",    "product_subcategory",    "product_modelname",    "customer_position",
      'customer_country.1', "response_corporate", "expected_timeline",
'nation','continent','lead_owner','idx_posi', 'conti_inquiry', 'idx_unit','bant_submit'
,'total_area'   ]
label_columns =list(set(cols)-set(columns_to_delete))

from category_encoders import CatBoostEncoder
enc = CatBoostEncoder(cols=label_columns)
enc.fit(train[label_columns], train['is_converted'])  # 'target'은 실제 데이터의 타겟 변수 이름에 맞게 변경
# 인코딩 적용
train[label_columns] = enc.transform(train[label_columns])
test[label_columns] = enc.transform(test[label_columns])

In [27]:
test = test.fillna(0)
train = train.fillna(0)
x = train.drop('is_converted', axis=1)
y = train.is_converted

In [30]:
def LGBM_skfold(zero_wei,one_wei,seed,x,y) :
     #Decisiontree에 대해서만 skfold 적용하는 함수 
    real_preds = []
    class_weight={0:zero_wei , 1:one_wei}
    model = LGBMClassifier(random_state=seed ,class_weight =class_weight, verbose=-1)
    min_features_to_select = 30
    step = 5
    selector = RFECV(model, step=step, cv=5, min_features_to_select=min_features_to_select)
    selector = selector.fit(x,y)
    features = selector.get_feature_names_out()
    train_k = x[features]
    Skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  
#     Skfold = StratifiedShuffleSplit(n_splits=10, random_state=42)  
    cv_precision_scores, cv_recall_scores, cv_confusion_matrices, cv_f1_scores, cv_roc_auc_scores, cv_TN = [],[],[],[],[],[]
    tt = []
    for train_index, test_index in Skfold.split(train_k, y):  
        x_train, x_test, y_train, y_test= train_k.iloc[train_index], train_k.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
        model.fit(x_train, y_train)
        pred = model.predict(x_test[features])
        test_pred = model.predict(test[features])
        real_preds.append(test_pred)
        
        pred_proba = model.predict_proba(x_test)[:, 1]  
        
        f1 = np.round(f1_score(y_test, pred, average='binary'), 4)  
        precision = np.round(precision_score(y_test, pred, average='binary'), 4)  
        recall = np.round(recall_score(y_test, pred, average='binary'), 4)  
        conf_matrix = confusion_matrix(y_test, pred)  
        roc_auc = roc_auc_score(y_test, pred_proba)  
        
        TN = conf_matrix[1][1]  # TN 값 저장
        ttone = conf_matrix[0][0]
        cv_TN.append(TN)  # TN 값 저장
        tt.append(ttone)
        cv_f1_scores.append(f1)  
        cv_precision_scores.append(precision)  
        cv_recall_scores.append(recall)  
        cv_confusion_matrices.append(conf_matrix)  
        cv_roc_auc_scores.append(roc_auc)  
        
    average_conf_matrix = np.mean(np.array(cv_confusion_matrices), axis=0)
    print(f"> F1 Score: {np.mean(cv_f1_scores)}, 정밀도: {np.mean(cv_precision_scores)}, 재현율: {np.mean(cv_recall_scores)}, ROC-AUC: {np.mean(cv_roc_auc_scores)}")
    print('> 평균 검증 오차행렬: \n', average_conf_matrix)  
    
    return real_preds, np.mean(cv_f1_scores), np.mean(cv_roc_auc_scores), np.mean(cv_TN) ,np.mean(tt) # TN 평균 값 리턴

f1_avg,roc_avg,tt=0,0,0
avg_get_1 =0
for seed in [5,11,30,322,8940]:
    _,f1,roc,ones,tts =LGBM_skfold(1,1,seed,x,y)
    f1_avg+= f1 
    roc_avg+= roc 
    avg_get_1+=ones
    tt+= tts 
print(f1_avg/5,roc_avg/5,avg_get_1/5, tt/5)


> F1 Score: 0.8976900000000001, 정밀도: 0.9324199999999999, 재현율: 0.86556, ROC-AUC: 0.9959010187998368
> 평균 검증 오차행렬: 
 [[5414.4   30.5]
 [  65.2  419.8]]
> F1 Score: 0.8976900000000001, 정밀도: 0.9324199999999999, 재현율: 0.86556, ROC-AUC: 0.9959010187998368
> 평균 검증 오차행렬: 
 [[5414.4   30.5]
 [  65.2  419.8]]
> F1 Score: 0.8976900000000001, 정밀도: 0.9324199999999999, 재현율: 0.86556, ROC-AUC: 0.9959010187998368
> 평균 검증 오차행렬: 
 [[5414.4   30.5]
 [  65.2  419.8]]
> F1 Score: 0.8976900000000001, 정밀도: 0.9324199999999999, 재현율: 0.86556, ROC-AUC: 0.9959010187998368
> 평균 검증 오차행렬: 
 [[5414.4   30.5]
 [  65.2  419.8]]
> F1 Score: 0.8976900000000001, 정밀도: 0.9324199999999999, 재현율: 0.86556, ROC-AUC: 0.9959010187998368
> 평균 검증 오차행렬: 
 [[5414.4   30.5]
 [  65.2  419.8]]
0.8976900000000001 0.9959010187998368 419.8 5414.4


In [ ]:
LGBM_preds,_,_,_,_= LGBM_skfold(1,1,3,x,y)

LGBM_predicts_array = np.array(LGBM_preds)
LGBM_final_prediction = np.mean(LGBM_predicts_array, axis=0)
LGBM_final_prediction = np.where(LGBM_final_prediction < 0.0000000001, 0, 1)

count_0 = np.size(np.where(LGBM_final_prediction == 0))
count_2 = np.size(np.where(LGBM_final_prediction >0))
count_1 = np.size(np.where(LGBM_final_prediction == 1))

# 각 값을 출력
print("Count of 0:", count_0)
print("Count of 1:", count_1)

> F1 Score: 0.89961, 정밀도: 0.9322399999999998, 재현율: 0.86928, ROC-AUC: 0.9959477578725165
> 평균 검증 오차행렬: 
 [[5414.2   30.7]
 [  63.4  421.6]]
Count of 0: 4584
Count of 1: 687


In [32]:
def XGB_skfold(zero_wei,one_wei,seed,x,y) :
     #Decisiontree에 대해서만 skfold 적용하는 함수 
    real_preds = []
    class_weight={0:zero_wei , 1:one_wei}
    model = XGBClassifier(random_state=seed ,class_weight =class_weight, verbose=-1)
    min_features_to_select = 30
    step = 5
    selector = RFECV(model, step=step, cv=5, min_features_to_select=min_features_to_select)
    selector = selector.fit(x,y)
    features = selector.get_feature_names_out()
    train_k = x[features]
    Skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  
#     Skfold = StratifiedShuffleSplit(n_splits=10, random_state=42)  
    cv_precision_scores, cv_recall_scores, cv_confusion_matrices, cv_f1_scores, cv_roc_auc_scores, cv_TN = [],[],[],[],[],[]
    tt = []
    for train_index, test_index in Skfold.split(train_k, y):  
        x_train, x_test, y_train, y_test= train_k.iloc[train_index], train_k.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
        model.fit(x_train, y_train)
        pred = model.predict(x_test[features])
        test_pred = model.predict(test[features])
        real_preds.append(test_pred)
        
        pred_proba = model.predict_proba(x_test)[:, 1]  
        
        f1 = np.round(f1_score(y_test, pred, average='binary'), 4)  
        precision = np.round(precision_score(y_test, pred, average='binary'), 4)  
        recall = np.round(recall_score(y_test, pred, average='binary'), 4)  
        conf_matrix = confusion_matrix(y_test, pred)  
        roc_auc = roc_auc_score(y_test, pred_proba)  
        
        TN = conf_matrix[1][1]  # TN 값 저장
        ttone = conf_matrix[0][0]
        cv_TN.append(TN)  # TN 값 저장
        tt.append(ttone)
        cv_f1_scores.append(f1)  
        cv_precision_scores.append(precision)  
        cv_recall_scores.append(recall)  
        cv_confusion_matrices.append(conf_matrix)  
        cv_roc_auc_scores.append(roc_auc)  
        
    average_conf_matrix = np.mean(np.array(cv_confusion_matrices), axis=0)
    print(f"> F1 Score: {np.mean(cv_f1_scores)}, 정밀도: {np.mean(cv_precision_scores)}, 재현율: {np.mean(cv_recall_scores)}, ROC-AUC: {np.mean(cv_roc_auc_scores)}")
    print('> 평균 검증 오차행렬: \n', average_conf_matrix)  
    
    return real_preds, np.mean(cv_f1_scores), np.mean(cv_roc_auc_scores), np.mean(cv_TN) ,np.mean(tt) # TN 평균 값 리턴

f1_avg,roc_avg,tt=0,0,0
avg_get_1 =0
for seed in [5,11,30,322,8940]:
    _,f1,roc,ones,tts =XGB_skfold(1,1,seed,x,y)
    f1_avg+= f1 
    roc_avg+= roc 
    avg_get_1+=ones
    tt+= tts 
print(f1_avg/5,roc_avg/5,avg_get_1/5, tt/5)


> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
0.8983099999999998 0.9959234930772787 421.7 5412.7


In [33]:
XGB_preds,_,_,_,_= XGB_skfold(1,1,100,x,y)

XGB_predicts_array = np.array(XGB_preds)
XGB_final_prediction = np.mean(XGB_predicts_array, axis=0)
XGB_final_prediction = np.where(XGB_final_prediction < 0.0000000001, 0, 1)

count_0 = np.size(np.where(XGB_final_prediction == 0))
count_2 = np.size(np.where(XGB_final_prediction >0))
count_1 = np.size(np.where(XGB_final_prediction == 1))

# 각 값을 출력
print("Count of 0:", count_0)
print("Count of 1:", count_1)

> F1 Score: 0.8983099999999998, 정밀도: 0.92917, 재현율: 0.8694900000000001, ROC-AUC: 0.9959234930772787
> 평균 검증 오차행렬: 
 [[5412.7   32.2]
 [  63.3  421.7]]
Count of 0: 4512
Count of 1: 759


In [34]:
def dtc_skfold(zero_wei,one_wei,seed,x,y) :
     #Decisiontree에 대해서만 skfold 적용하는 함수 
    real_preds = []
    class_weight={0:zero_wei , 1:one_wei}
    model = DecisionTreeClassifier(random_state=seed ,class_weight =class_weight)
    min_features_to_select = 30
    step = 5
    selector = RFECV(model, step=step, cv=5, min_features_to_select=min_features_to_select)
    selector = selector.fit(x,y)
    features = selector.get_feature_names_out()
    train_k = x[features]
    Skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  
#     Skfold = StratifiedShuffleSplit(n_splits=10, random_state=42)  
    cv_precision_scores, cv_recall_scores, cv_confusion_matrices, cv_f1_scores, cv_roc_auc_scores, cv_TN = [],[],[],[],[],[]
    tt = []
    for train_index, test_index in Skfold.split(train_k, y):  
        x_train, x_test, y_train, y_test= train_k.iloc[train_index], train_k.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
        model.fit(x_train, y_train)
        pred = model.predict(x_test[features])
        test_pred = model.predict(test[features])
        real_preds.append(test_pred)
        
        pred_proba = model.predict_proba(x_test)[:, 1]  
        
        f1 = np.round(f1_score(y_test, pred, average='binary'), 4)  
        precision = np.round(precision_score(y_test, pred, average='binary'), 4)  
        recall = np.round(recall_score(y_test, pred, average='binary'), 4)  
        conf_matrix = confusion_matrix(y_test, pred)  
        roc_auc = roc_auc_score(y_test, pred_proba)  
        
        TN = conf_matrix[1][1]  # TN 값 저장
        ttone = conf_matrix[0][0]
        cv_TN.append(TN)  # TN 값 저장
        tt.append(ttone)
        cv_f1_scores.append(f1)  
        cv_precision_scores.append(precision)  
        cv_recall_scores.append(recall)  
        cv_confusion_matrices.append(conf_matrix)  
        cv_roc_auc_scores.append(roc_auc)  
        
    average_conf_matrix = np.mean(np.array(cv_confusion_matrices), axis=0)
    print(f"> F1 Score: {np.mean(cv_f1_scores)}, 정밀도: {np.mean(cv_precision_scores)}, 재현율: {np.mean(cv_recall_scores)}, ROC-AUC: {np.mean(cv_roc_auc_scores)}")
    print('> 평균 검증 오차행렬: \n', average_conf_matrix)  
    
    return real_preds, np.mean(cv_f1_scores), np.mean(cv_roc_auc_scores), np.mean(cv_TN) ,np.mean(tt) # TN 평균 값 리턴

f1_avg,roc_avg,tt=0,0,0
avg_get_1 =0
for seed in [5,11,30,322,8940]:
    _,f1,roc,ones,tts =dtc_skfold(1,1,seed,x,y)
    f1_avg+= f1 
    roc_avg+= roc 
    avg_get_1+=ones
    tt+= tts 
print(f1_avg/5,roc_avg/5,avg_get_1/5, tt/5)



> F1 Score: 0.86432, 정밀도: 0.86097, 재현율: 0.86784, ROC-AUC: 0.9285443130835341
> 평균 검증 오차행렬: 
 [[5376.8   68.1]
 [  64.1  420.9]]
> F1 Score: 0.8627499999999999, 정밀도: 0.85968, 재현율: 0.8659699999999999, ROC-AUC: 0.9270518251771189
> 평균 검증 오차행렬: 
 [[5376.2   68.7]
 [  65.   420. ]]
> F1 Score: 0.8615999999999999, 정밀도: 0.85659, 재현율: 0.8668099999999999, ROC-AUC: 0.9277059579096838
> 평균 검증 오차행렬: 
 [[5374.4   70.5]
 [  64.6  420.4]]
> F1 Score: 0.86265, 정밀도: 0.8619200000000001, 재현율: 0.86351, ROC-AUC: 0.9261540858927282
> 평균 검증 오차행렬: 
 [[5377.7   67.2]
 [  66.2  418.8]]
> F1 Score: 0.86147, 정밀도: 0.8597700000000001, 재현율: 0.8633000000000001, ROC-AUC: 0.9258440671368303
> 평균 검증 오차행렬: 
 [[5376.5   68.4]
 [  66.3  418.7]]
0.8625579999999999 0.9270600498399789 419.75999999999993 5376.32


In [35]:
dtc_preds,_,_,_,_= dtc_skfold(1,1,3,x,y)

dtc_predicts_array = np.array(dtc_preds)
dtc_final_prediction = np.mean(dtc_predicts_array, axis=0)
dtc_final_prediction = np.where(dtc_final_prediction < 0.1, 0, 1)

count_0 = np.size(np.where(dtc_final_prediction == 0))
count_2 = np.size(np.where(dtc_final_prediction >0))
count_1 = np.size(np.where(dtc_final_prediction == 1))

# 각 값을 출력
print("Count of 0:", count_0)
print("Count of 1:", count_1)

> F1 Score: 0.86281, 정밀도: 0.8603999999999999, 재현율: 0.8653599999999999, ROC-AUC: 0.9272941284977818
> 평균 검증 오차행렬: 
 [[5376.7   68.2]
 [  65.3  419.7]]
Count of 0: 3098
Count of 1: 2173


In [36]:
# seed(3)에 대해서만 제출 할 때 
sub=pd.read_csv('submission.csv')
sub['is_converted']= dtc_final_prediction
sub.to_csv('RFECV.csv',index= False)

In [ ]:
dtc_preds,_,_,_,_= dtc_skfold(1,1,3,x,y)

dtc_predicts_array = np.array(dtc_preds)
dtc_final_prediction = np.mean(dtc_predicts_array, axis=0)
dtc_final_prediction = np.where(dtc_final_prediction < 0.1, 0, 1)

count_0 = np.size(np.where(dtc_final_prediction == 0))
count_2 = np.size(np.where(dtc_final_prediction >0))
count_1 = np.size(np.where(dtc_final_prediction == 1))

# 각 값을 출력
print("Count of 0:", count_0)
print("Count of 1:", count_1)

> F1 Score: 0.86283, 정밀도: 0.8610800000000001, 재현율: 0.8647500000000001, ROC-AUC: 0.9276289726627786
> 평균 검증 오차행렬: 
 [[5377.1   67.8]
 [  65.6  419.4]]
Count of 0: 3056
Count of 1: 2215


In [ ]:
len(test)

5271

In [ ]:
sum(LGBM_final_prediction)

687

In [ ]:
LGBM_final_prediction

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
sum(XGB_final_prediction)

775

In [ ]:
XGB_final_prediction

array([0, 1, 0, ..., 0, 0, 1])

In [ ]:
con_true = 0
for i in range(len(XGB_final_prediction)):
    if XGB_final_prediction[i] == 1 and dtc_final_prediction[i] == 1:
        con_true += 1

In [ ]:
con_true

758

In [ ]:
con_true = 0
for i in range(len(XGB_final_prediction)):
    if LGBM_final_prediction[i] == 1 and dtc_final_prediction[i] == 1:
        con_true += 1

In [ ]:
con_true

671

In [ ]:
for idx in range(len(dtc_final_prediction)):
    if dtc_final_prediction[idx] == 0:
        change = 0
        if LGBM_final_prediction[idx] == 1 or XGB_final_prediction[idx] == 1:
            change = 1
        dtc_final_prediction[idx] = change

print(sum(dtc_final_prediction))
         

2240


In [ ]:
# seed(3)에 대해서만 제출 할 때 
sub=pd.read_csv('submission.csv')
sub['is_converted']= dtc_final_prediction
sub.to_csv('XGB_LGBM_DTC.csv',index= False)